In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# COMPLETE EMOTION RECOGNITION TRAINING
# Using Your emotions-dataset
# ══════════════════════════════════════════════════════════════════════════════

import os
import torch
import numpy as np
import librosa
from pathlib import Path
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    Wav2Vec2ForSequenceClassification,
    Wav2Vec2FeatureExtractor,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# ══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION
# ══════════════════════════════════════════════════════════════════════════════

MODEL_NAME = "facebook/wav2vec2-xls-r-300m"
OUTPUT_DIR = "./emotion-recognition-model"
NUM_EPOCHS = 15
BATCH_SIZE = 4
LEARNING_RATE = 2e-5
MAX_LENGTH = 5 * 16000  # 5 seconds at 16kHz

# Emotion labels based on your dataset structure
EMOTION_LABELS = {
    "angry": 0,
    "happy": 1,
    "hesitant": 2,
    "interested": 3,
    "neutral": 4
}

LABEL_TO_ID = EMOTION_LABELS
ID_TO_LABEL = {v: k for k, v in EMOTION_LABELS.items()}

print(f"🎯 Emotion Labels: {EMOTION_LABELS}")
print(f"📊 Number of classes: {len(EMOTION_LABELS)}")

# ══════════════════════════════════════════════════════════════════════════════
# DATA LOADING
# ══════════════════════════════════════════════════════════════════════════════

def load_audio_file(file_path, target_sr=16000):
    """Load and preprocess audio file"""
    try:
        audio, sr = librosa.load(file_path, sr=target_sr)
        return audio
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def load_dataset_from_folders(base_path):
    """
    Load dataset from your Kaggle folder structure:
    /kaggle/input/emotions-dataset/Emotions/Angry/*.wav
    /kaggle/input/emotions-dataset/Emotions/Happy/*.wav
    etc.
    """
    data = []
    emotions_path = base_path / "Emotions"
    
    print(f"\n📂 Loading data from: {emotions_path}")
    
    # Map folder names to standardized emotion labels
    folder_to_emotion = {
        "Angry": "angry",
        "Happy": "happy",
        "Hesitant": "hesitant",
        "Interested": "interested",
        "Neutral": "neutral"
    }
    
    for folder_name, emotion in folder_to_emotion.items():
        emotion_folder = emotions_path / folder_name
        
        if not emotion_folder.exists():
            print(f"⚠️  Warning: {emotion_folder} not found!")
            continue
        
        # Find all audio files (wav, mp3, etc.)
        audio_files = list(emotion_folder.glob("*.wav")) + \
                     list(emotion_folder.glob("*.mp3")) + \
                     list(emotion_folder.glob("*.m4a"))
        
        print(f"  {emotion}: {len(audio_files)} files")
        
        for audio_file in audio_files:
            data.append({
                "path": str(audio_file),
                "emotion": emotion,
                "label": EMOTION_LABELS[emotion]
            })
    
    print(f"\n✅ Total samples loaded: {len(data)}")
    return data

# Load your dataset
DATASET_PATH = Path("/kaggle/input/emotions-dataset")
all_data = load_dataset_from_folders(DATASET_PATH)

# Split into train/val/test
train_data, temp_data = train_test_split(all_data, test_size=0.3, random_state=42, stratify=[d["label"] for d in all_data])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=[d["label"] for d in temp_data])

print(f"\n📊 Dataset splits:")
print(f"  Train: {len(train_data)} samples")
print(f"  Validation: {len(val_data)} samples")
print(f"  Test: {len(test_data)} samples")

# ══════════════════════════════════════════════════════════════════════════════
# FEATURE EXTRACTION
# ══════════════════════════════════════════════════════════════════════════════

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    """Process audio files for the model"""
    audio_arrays = []
    
    for path in examples["path"]:
        audio = load_audio_file(path)
        if audio is not None:
            # Pad or truncate to MAX_LENGTH
            if len(audio) > MAX_LENGTH:
                audio = audio[:MAX_LENGTH]
            else:
                audio = np.pad(audio, (0, MAX_LENGTH - len(audio)))
            audio_arrays.append(audio)
        else:
            # If file fails to load, use silence
            audio_arrays.append(np.zeros(MAX_LENGTH))
    
    # Extract features
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=16000,
        padding=True,
        return_tensors="pt"
    )
    
    return {
        "input_values": inputs.input_values,
        "labels": examples["label"]
    }

# Create HuggingFace datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

# Apply preprocessing
print("\n🔄 Preprocessing datasets...")
dataset_dict = dataset_dict.map(
    preprocess_function,
    batched=True,
    batch_size=8,
    remove_columns=["path", "emotion"]
)

print("✅ Preprocessing complete!")

# ══════════════════════════════════════════════════════════════════════════════
# MODEL SETUP
# ══════════════════════════════════════════════════════════════════════════════

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(EMOTION_LABELS),
    label2id=LABEL_TO_ID,
    id2label=ID_TO_LABEL,
    ignore_mismatched_sizes=True
)

# Freeze feature extractor
model.freeze_feature_encoder()

print(f"\n🤖 Model loaded: {MODEL_NAME}")
print(f"📊 Number of parameters: {model.num_parameters():,}")

# ══════════════════════════════════════════════════════════════════════════════
# TRAINING CONFIGURATION
# ══════════════════════════════════════════════════════════════════════════════

def compute_metrics(eval_pred):
    """Compute accuracy and F1 score"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        "accuracy": acc,
        "f1": f1
    }

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    fp16=True,  # Use mixed precision for faster training
    save_total_limit=2,
    report_to=["tensorboard"]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    compute_metrics=compute_metrics,
)

print("\n🚀 Starting training...")
print(f"⏱️  Epochs: {NUM_EPOCHS}")
print(f"📦 Batch size: {BATCH_SIZE}")
print(f"📈 Learning rate: {LEARNING_RATE}")

# ══════════════════════════════════════════════════════════════════════════════
# TRAIN THE MODEL
# ══════════════════════════════════════════════════════════════════════════════

trainer.train()

print("\n✅ Training complete!")

# ══════════════════════════════════════════════════════════════════════════════
# EVALUATION
# ══════════════════════════════════════════════════════════════════════════════

print("\n📊 Evaluating on test set...")
test_results = trainer.evaluate(dataset_dict["test"])

print("\n" + "="*60)
print("FINAL TEST RESULTS")
print("="*60)
for key, value in test_results.items():
    print(f"{key}: {value:.4f}")
print("="*60)

# ══════════════════════════════════════════════════════════════════════════════
# SAVE MODEL
# ══════════════════════════════════════════════════════════════════════════════






final_model_path = f"{OUTPUT_DIR}/final"
trainer.save_model(final_model_path)
feature_extractor.save_pretrained(final_model_path)

print(f"\n💾 Model saved to: {final_model_path}")
print("\n🎉 All done! Your emotion recognition model is ready!")
 """
Check if your model was saved correctly
"""
import os
from pathlib import Path

print("Checking model directory...")
print("="*60)

# Check main model directory
model_dir = Path("/kaggle/working/emotion-recognition-model")

if model_dir.exists():
    print(f"✅ Directory exists: {model_dir}")
    print(f"\nContents of {model_dir}:")
    print("-"*60)
    
    for item in sorted(model_dir.iterdir()):
        if item.is_dir():
            print(f"  📁 {item.name}/")
            # Check subdirectories
            for subitem in sorted(item.iterdir())[:5]:  # Show first 5 items
                print(f"     - {subitem.name}")
            if len(list(item.iterdir())) > 5:
                print(f"     ... and {len(list(item.iterdir())) - 5} more files")
        else:
            size = item.stat().st_size / (1024 * 1024)  # Size in MB
            print(f"  📄 {item.name} ({size:.2f} MB)")
    
    # Check for required files
    print("\n" + "="*60)
    print("Checking for required model files:")
    print("-"*60)
    
    required_files = [
        "config.json",
        "preprocessor_config.json",
        "model.safetensors",  # or pytorch_model.bin
    ]
    
    for file in required_files:
        file_path = model_dir / file
        if file_path.exists():
            print(f"  ✅ {file}")
        else:
            # Check alternative names
            if file == "model.safetensors":
                alt_path = model_dir / "pytorch_model.bin"
                if alt_path.exists():
                    print(f"  ✅ pytorch_model.bin (alternative)")
                else:
                    print(f"  ❌ {file} (MISSING!)")
            else:
                print(f"  ❌ {file} (MISSING!)")
    
    # Check if there's a 'final' subdirectory
    final_dir = model_dir / "final"
    if final_dir.exists():
        print(f"\n✅ Found 'final' subdirectory!")
        print(f"\nContents of {final_dir}:")
        print("-"*60)
        for item in sorted(final_dir.iterdir()):
            if item.is_file():
                size = item.stat().st_size / (1024 * 1024)
                print(f"  📄 {item.name} ({size:.2f} MB)")
    else:
        print(f"\n⚠️  No 'final' subdirectory found")
    
else:
    print(f"❌ Directory does NOT exist: {model_dir}")
    print("\n⚠️  This means the model was NOT saved!")
    print("\nPossible reasons:")
    print("  1. Training did not complete successfully")
    print("  2. Training is still running")
    print("  3. Model was saved to a different location")
    
    # Check working directory
    print(f"\nChecking /kaggle/working/ directory:")
    print("-"*60)
    working_dir = Path("/kaggle/working")
    if working_dir.exists():
        for item in sorted(working_dir.iterdir()):
            if item.is_dir():
                print(f"  📁 {item.name}/")
            else:
                size = item.stat().st_size / (1024 * 1024)
                print(f"  📄 {item.name} ({size:.2f} MB)")

print("\n" + "="*60)
print("CONCLUSION:")
print("="*60)

model_path = Path("/kaggle/working/emotion-recognition-model")
final_path = model_path / "final"

if final_path.exists() and (final_path / "config.json").exists():
    print("✅ Model appears to be saved correctly!")
    print(f"   Use this path: {final_path}")
elif model_path.exists() and (model_path / "config.json").exists():
    print("✅ Model is saved in main directory!")
    print(f"   Use this path: {model_path}")
else:
    print("❌ Model was NOT saved properly!")
    print("   You need to run the training script again.")





    """
Test 10 Voice Notes from Each Emotion Category
This script samples 10 VNs from each emotion (angry, happy, hesitant, interested, neutral)
and evaluates them using your trained model.
"""

import os
import torch
import numpy as np
import librosa
from pathlib import Path
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import pandas as pd
import random
import warnings
warnings.filterwarnings('ignore')

# ══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION
# ══════════════════════════════════════════════════════════════════════════════

MODEL_PATH = "./emotion-recognition-model/final"  # Update this path if needed
DATASET_PATH = Path("/kaggle/input/emotions-dataset")  # Update this path if needed
MAX_LENGTH = 5 * 16000  # 5 seconds at 16kHz
SAMPLES_PER_EMOTION = 10

# Emotion labels
EMOTION_LABELS = {
    "angry": 0,
    "happy": 1,
    "hesitant": 2,
    "interested": 3,
    "neutral": 4
}

ID_TO_LABEL = {v: k for k, v in EMOTION_LABELS.items()}

print("=" * 80)
print("TESTING 10 VNs FROM EACH EMOTION CATEGORY")
print("=" * 80)

# ══════════════════════════════════════════════════════════════════════════════
# LOAD MODEL
# ══════════════════════════════════════════════════════════════════════════════

print("\n📦 Loading model and feature extractor...")
try:
    model = Wav2Vec2ForSequenceClassification.from_pretrained(MODEL_PATH)
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_PATH)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print(f"✅ Model loaded successfully on {device}")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\nMake sure you've trained the model first and MODEL_PATH is correct!")
    exit(1)

# ══════════════════════════════════════════════════════════════════════════════
# LOAD AUDIO SAMPLES
# ══════════════════════════════════════════════════════════════════════════════

def load_audio_file(file_path, target_sr=16000):
    """Load and preprocess audio file"""
    try:
        audio, sr = librosa.load(file_path, sr=target_sr)
        return audio
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def get_samples_per_emotion(base_path, samples_per_emotion=10):
    """
    Sample 10 audio files from each emotion folder
    """
    emotions_path = base_path / "Emotions"
    
    # Map folder names to standardized emotion labels
    folder_to_emotion = {
        "Angry": "angry",
        "Happy": "happy",
        "Hesitant": "hesitant",
        "Interested": "interested",
        "Neutral": "neutral"
    }
    
    test_samples = []
    
    print(f"\n📂 Sampling {samples_per_emotion} files from each emotion folder...")
    print("-" * 80)
    
    for folder_name, emotion in folder_to_emotion.items():
        emotion_folder = emotions_path / folder_name
        
        if not emotion_folder.exists():
            print(f"⚠️  Warning: {emotion_folder} not found!")
            continue
        
        # Find all audio files
        audio_files = list(emotion_folder.glob("*.wav")) + \
                     list(emotion_folder.glob("*.mp3")) + \
                     list(emotion_folder.glob("*.m4a"))
        
        # Randomly sample files
        if len(audio_files) >= samples_per_emotion:
            sampled_files = random.sample(audio_files, samples_per_emotion)
        else:
            print(f"⚠️  Warning: Only {len(audio_files)} files found for {emotion}, using all of them")
            sampled_files = audio_files
        
        print(f"  {emotion}: {len(sampled_files)} files sampled")
        
        for audio_file in sampled_files:
            test_samples.append({
                "path": str(audio_file),
                "emotion": emotion,
                "label": EMOTION_LABELS[emotion],
                "filename": audio_file.name
            })
    
    print(f"\n✅ Total test samples: {len(test_samples)}")
    return test_samples

# Load test samples
random.seed(42)  # For reproducibility
test_samples = get_samples_per_emotion(DATASET_PATH, SAMPLES_PER_EMOTION)

# ══════════════════════════════════════════════════════════════════════════════
# RUN PREDICTIONS
# ══════════════════════════════════════════════════════════════════════════════

print("\n🔄 Running predictions...")
print("-" * 80)

predictions = []
true_labels = []
results_details = []

for idx, sample in enumerate(test_samples):
    # Load audio
    audio = load_audio_file(sample["path"])
    
    if audio is None:
        print(f"⚠️  Skipping {sample['filename']} (failed to load)")
        continue
    
    # Pad or truncate
    if len(audio) > MAX_LENGTH:
        audio = audio[:MAX_LENGTH]
    else:
        audio = np.pad(audio, (0, MAX_LENGTH - len(audio)))
    
    # Extract features
    inputs = feature_extractor(
        audio,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )
    
    # Move to device
    input_values = inputs.input_values.to(device)
    
    # Predict
    with torch.no_grad():
        logits = model(input_values).logits
        predicted_id = torch.argmax(logits, dim=-1).item()
    
    predicted_emotion = ID_TO_LABEL[predicted_id]
    true_emotion = sample["emotion"]
    
    predictions.append(predicted_id)
    true_labels.append(sample["label"])
    
    # Store detailed results
    results_details.append({
        "filename": sample["filename"],
        "true_emotion": true_emotion,
        "predicted_emotion": predicted_emotion,
        "correct": predicted_emotion == true_emotion
    })
    
    # Progress indicator
    if (idx + 1) % 10 == 0:
        print(f"  Processed {idx + 1}/{len(test_samples)} samples...")

print(f"\n✅ Predictions complete!")

# ══════════════════════════════════════════════════════════════════════════════
# CALCULATE METRICS
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("OVERALL RESULTS")
print("=" * 80)

accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Overall Accuracy: {accuracy:.4f} ({accuracy * 100:.2f}%)")
print(f"Overall F1 Score: {f1:.4f}")

# ══════════════════════════════════════════════════════════════════════════════
# PER-EMOTION BREAKDOWN
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("PER-EMOTION ACCURACY")
print("=" * 80)

emotion_stats = {}
for emotion in EMOTION_LABELS.keys():
    emotion_results = [r for r in results_details if r["true_emotion"] == emotion]
    if emotion_results:
        correct = sum(1 for r in emotion_results if r["correct"])
        total = len(emotion_results)
        accuracy_pct = (correct / total) * 100
        emotion_stats[emotion] = {
            "correct": correct,
            "total": total,
            "accuracy": accuracy_pct
        }
        print(f"  {emotion.capitalize():12s}: {correct}/{total} correct ({accuracy_pct:.2f}%)")

# ══════════════════════════════════════════════════════════════════════════════
# CONFUSION MATRIX
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("CONFUSION MATRIX")
print("=" * 80)

cm = confusion_matrix(true_labels, predictions)
emotion_names = [ID_TO_LABEL[i] for i in range(len(EMOTION_LABELS))]

# Print confusion matrix
print("\n" + " " * 15 + "Predicted")
print(" " * 12, end="")
for name in emotion_names:
    print(f"{name[:8]:>10s}", end="")
print("\nActual")

for i, name in enumerate(emotion_names):
    print(f"{name[:8]:>10s}", end="  ")
    for j in range(len(emotion_names)):
        print(f"{cm[i][j]:>10d}", end="")
    print()

# ══════════════════════════════════════════════════════════════════════════════
# DETAILED CLASSIFICATION REPORT
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 80)

print(classification_report(
    true_labels, 
    predictions, 
    target_names=emotion_names,
    digits=4
))

# ══════════════════════════════════════════════════════════════════════════════
# SAVE DETAILED RESULTS TO CSV
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("SAVING RESULTS")
print("=" * 80)

# Create DataFrame
df_results = pd.DataFrame(results_details)

# Save to CSV
output_file = "test_results_10_per_emotion.csv"
df_results.to_csv(output_file, index=False)
print(f"✅ Detailed results saved to: {output_file}")

# ══════════════════════════════════════════════════════════════════════════════
# MISCLASSIFIED SAMPLES (WRONG PREDICTIONS)
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("WRONG PREDICTIONS - MODEL ERRORS")
print("=" * 80)

misclassified = df_results[df_results["correct"] == False]
if len(misclassified) > 0:
    print(f"\n❌ Total WRONG predictions: {len(misclassified)}/{len(df_results)}")
    print(f"   Error Rate: {(len(misclassified)/len(df_results)*100):.2f}%")
    print("\n" + "-" * 80)
    print("MODEL WAS WRONG - Here are the mistakes:")
    print("-" * 80)
    
    for idx, row in misclassified.iterrows():
        print(f"\n❌ File: {row['filename']}")
        print(f"   ➤ Model predicted: {row['predicted_emotion'].upper()} (WRONG!)")
        print(f"   ✓ Correct answer was: {row['true_emotion'].upper()}")
        print(f"   → Model confused '{row['true_emotion']}' for '{row['predicted_emotion']}'")
    
    # Group errors by confusion type
    print("\n" + "=" * 80)
    print("ERROR PATTERNS - What the model got confused about")
    print("=" * 80)
    
    error_patterns = {}
    for idx, row in misclassified.iterrows():
        key = f"{row['true_emotion']} → {row['predicted_emotion']}"
        if key not in error_patterns:
            error_patterns[key] = []
        error_patterns[key].append(row['filename'])
    
    for pattern, files in sorted(error_patterns.items(), key=lambda x: len(x[1]), reverse=True):
        true_em, pred_em = pattern.split(' → ')
        print(f"\n❌ Confused {true_em.upper()} as {pred_em.upper()}: {len(files)} times")
        for file in files:
            print(f"   - {file}")
else:
    print("\n🎉 Perfect! No misclassifications!")
    print("   The model got ALL predictions correct!")

# ══════════════════════════════════════════════════════════════════════════════
# SUMMARY
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("FINAL SUMMARY")
print("=" * 80)

correct_predictions = len(df_results) - len(misclassified)
print(f"\n📊 RESULTS:")
print(f"   ✅ Correct predictions: {correct_predictions}/{len(test_samples)}")
print(f"   ❌ Wrong predictions: {len(misclassified)}/{len(test_samples)}")
print(f"   📈 Accuracy: {accuracy * 100:.2f}%")
print(f"   📊 F1 Score: {f1:.4f}")

if len(misclassified) > 0:
    print(f"\n❌ MODEL MISTAKES RECAP:")
    print(f"   The model was WRONG on {len(misclassified)} predictions:")
    for idx, row in misclassified.iterrows():
        print(f"   • {row['filename'][:35]:35s} - Predicted: {row['predicted_emotion']:10s} | Should be: {row['true_emotion']:10s}")

print(f"\n💾 All results saved to: {output_file}")
print("\n" + "=" * 80)
print("🎉 Testing complete!")
print("=" * 80)


2026-01-29 17:15:36.709056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769706936.903692      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769706936.958986      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769706937.422881      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769706937.422929      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769706937.422932      55 computation_placer.cc:177] computation placer alr

🎯 Emotion Labels: {'angry': 0, 'happy': 1, 'hesitant': 2, 'interested': 3, 'neutral': 4}
📊 Number of classes: 5

📂 Loading data from: /kaggle/input/emotions-dataset/Emotions
  angry: 499 files
  happy: 500 files
  hesitant: 502 files
  interested: 498 files
  neutral: 500 files

✅ Total samples loaded: 2499

📊 Dataset splits:
  Train: 1749 samples
  Validation: 375 samples
  Test: 375 samples


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]


🔄 Preprocessing datasets...


Map:   0%|          | 0/1749 [00:00<?, ? examples/s]

...

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🤖 Model loaded: facebook/wav2vec2-xls-r-300m
📊 Number of parameters: 315,702,405

🚀 Starting training...
⏱️  Epochs: 15
📦 Batch size: 4
📈 Learning rate: 2e-05


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.602900,1.599049,0.197333,0.070883
2,1.356900,1.230649,0.597333,0.548685
3,0.908100,0.799255,0.749333,0.710771
4,0.798300,0.646883,0.792000,0.788147
5,0.577300,0.502910,0.810667,0.802229
6,0.533400,0.517490,0.805333,0.804103
7,0.514800,0.561902,0.786667,0.786075
8,0.387700,0.567250,0.762667,0.761251
9,0.215400,0.358186,0.880000,0.880687
10,0.220000,0.391238,0.872000,0.870476



✅ Training complete!

📊 Evaluating on test set...



FINAL TEST RESULTS
eval_loss: 0.4150
eval_accuracy: 0.8613
eval_f1: 0.8641
eval_runtime: 36.5548
eval_samples_per_second: 10.2590
eval_steps_per_second: 1.2860
epoch: 15.0000

💾 Model saved to: ./emotion-recognition-model/final

🎉 All done! Your emotion recognition model is ready!


In [ ]:
"""
Check if your model was saved correctly
"""
import os
from pathlib import Path

print("Checking model directory...")
print("="*60)

# Check main model directory
model_dir = Path("/kaggle/working/emotion-recognition-model")

if model_dir.exists():
    print(f"✅ Directory exists: {model_dir}")
    print(f"\nContents of {model_dir}:")
    print("-"*60)
    
    for item in sorted(model_dir.iterdir()):
        if item.is_dir():
            print(f"  📁 {item.name}/")
            # Check subdirectories
            for subitem in sorted(item.iterdir())[:5]:  # Show first 5 items
                print(f"     - {subitem.name}")
            if len(list(item.iterdir())) > 5:
                print(f"     ... and {len(list(item.iterdir())) - 5} more files")
        else:
            size = item.stat().st_size / (1024 * 1024)  # Size in MB
            print(f"  📄 {item.name} ({size:.2f} MB)")
    
    # Check for required files
    print("\n" + "="*60)
    print("Checking for required model files:")
    print("-"*60)
    
    required_files = [
        "config.json",
        "preprocessor_config.json",
        "model.safetensors",  # or pytorch_model.bin
    ]
    
    for file in required_files:
        file_path = model_dir / file
        if file_path.exists():
            print(f"  ✅ {file}")
        else:
            # Check alternative names
            if file == "model.safetensors":
                alt_path = model_dir / "pytorch_model.bin"
                if alt_path.exists():
                    print(f"  ✅ pytorch_model.bin (alternative)")
                else:
                    print(f"  ❌ {file} (MISSING!)")
            else:
                print(f"  ❌ {file} (MISSING!)")
    
    # Check if there's a 'final' subdirectory
    final_dir = model_dir / "final"
    if final_dir.exists():
        print(f"\n✅ Found 'final' subdirectory!")
        print(f"\nContents of {final_dir}:")
        print("-"*60)
        for item in sorted(final_dir.iterdir()):
            if item.is_file():
                size = item.stat().st_size / (1024 * 1024)
                print(f"  📄 {item.name} ({size:.2f} MB)")
    else:
        print(f"\n⚠️  No 'final' subdirectory found")
    
else:
    print(f"❌ Directory does NOT exist: {model_dir}")
    print("\n⚠️  This means the model was NOT saved!")
    print("\nPossible reasons:")
    print("  1. Training did not complete successfully")
    print("  2. Training is still running")
    print("  3. Model was saved to a different location")
    
    # Check working directory
    print(f"\nChecking /kaggle/working/ directory:")
    print("-"*60)
    working_dir = Path("/kaggle/working")
    if working_dir.exists():
        for item in sorted(working_dir.iterdir()):
            if item.is_dir():
                print(f"  📁 {item.name}/")
            else:
                size = item.stat().st_size / (1024 * 1024)
                print(f"  📄 {item.name} ({size:.2f} MB)")

print("\n" + "="*60)
print("CONCLUSION:")
print("="*60)

model_path = Path("/kaggle/working/emotion-recognition-model")
final_path = model_path / "final"

if final_path.exists() and (final_path / "config.json").exists():
    print("✅ Model appears to be saved correctly!")
    print(f"   Use this path: {final_path}")
elif model_path.exists() and (model_path / "config.json").exists():
    print("✅ Model is saved in main directory!")
    print(f"   Use this path: {model_path}")
else:
    print("❌ Model was NOT saved properly!")
    print("   You need to run the training script again.")


    """
Test Unknown Dataset Voice Notes
Testing 3 external VNs: angry.ogg, happy.ogg, netural.ogg
"""

import os
import torch
import numpy as np
import librosa
from pathlib import Path
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import warnings
warnings.filterwarnings('ignore')

# ══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION
# ══════════════════════════════════════════════════════════════════════════════

MODEL_PATH = "./emotion-recognition-model/final"  # Update this path if needed
# Try multiple possible locations for the dataset
POSSIBLE_PATHS = [
    "./unknown-dataset",                              # Local directory
    "/kaggle/input/unknown-datasettt/unknown-dataset", # Kaggle input
    "/kaggle/working/unknown-dataset"                 # Kaggle working
]

# Find the correct path
AUDIO_FOLDER = None
for path in POSSIBLE_PATHS:
    if os.path.exists(path):
        AUDIO_FOLDER = path
        break

if AUDIO_FOLDER is None:
    print("❌ Error: Could not find unknown-dataset folder!")
    print("Searched in:")
    for path in POSSIBLE_PATHS:
        print(f"  - {path}")
    exit(1)

MAX_LENGTH = 5 * 16000  # 5 seconds at 16kHz

# Emotion labels
EMOTION_LABELS = {
    "angry": 0,
    "happy": 1,
    "hesitant": 2,
    "interested": 3,
    "neutral": 4
}

ID_TO_LABEL = {v: k for k, v in EMOTION_LABELS.items()}

print("=" * 80)
print("TESTING UNKNOWN DATASET VOICE NOTES")
print("=" * 80)
print(f"\n📍 Found dataset at: {AUDIO_FOLDER}")
print("\n📂 Dataset files:")
print("   1. angry (1).wav")
print("   2. happy (1).wav")
print("   3. happyb.wav")
print("   4. netural.wav (neutral)")
print("\n🎯 Goal: See what the model predicts for each file")
print("=" * 80)

# ══════════════════════════════════════════════════════════════════════════════
# LOAD MODEL
# ══════════════════════════════════════════════════════════════════════════════

print("\n📦 Loading trained model...")
try:
    model = Wav2Vec2ForSequenceClassification.from_pretrained(MODEL_PATH)
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_PATH)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print(f"✅ Model loaded successfully on {device}")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\nMake sure:")
    print("  1. You've trained the model first")
    print("  2. MODEL_PATH is correct")
    print(f"  3. The model exists at: {MODEL_PATH}")
    exit(1)

# ══════════════════════════════════════════════════════════════════════════════
# LOAD AUDIO FUNCTION
# ══════════════════════════════════════════════════════════════════════════════

def load_audio_file(file_path, target_sr=16000):
    """Load and preprocess audio file"""
    try:
        audio, sr = librosa.load(file_path, sr=target_sr)
        return audio
    except Exception as e:
        print(f"❌ Error loading {file_path}: {e}")
        return None

# ══════════════════════════════════════════════════════════════════════════════
# TEST FILES SETUP
# ══════════════════════════════════════════════════════════════════════════════

# Define test files with expected emotion (what the filename suggests)
test_files = [
    {"filename": "angry (1).wav", "expected": "angry"},
    {"filename": "happy (1).wav", "expected": "happy"},
    {"filename": "happyb.wav", "expected": "happy"},  # Another happy sample
    {"filename": "netural.wav", "expected": "neutral"},  # Note: typo in filename
]

results = []

# ══════════════════════════════════════════════════════════════════════════════
# PREDICT EMOTION FOR EACH FILE
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("RUNNING PREDICTIONS")
print("=" * 80)

for i, file_info in enumerate(test_files, 1):
    filename = file_info["filename"]
    expected_emotion = file_info["expected"]
    file_path = os.path.join(AUDIO_FOLDER, filename)
    
    print(f"\n{'=' * 80}")
    print(f"TESTING FILE {i}/4: {filename}")
    print(f"Expected emotion (from filename): {expected_emotion.upper()}")
    print('=' * 80)
    
    # Check if file exists
    if not os.path.exists(file_path):
        print(f"  ❌ File not found: {file_path}")
        results.append({
            "filename": filename,
            "expected": expected_emotion,
            "status": "FILE NOT FOUND",
            "predicted_emotion": "N/A",
            "confidence": 0.0,
            "correct": False
        })
        continue
    
    # Load audio
    print("  📥 Loading audio file...")
    audio = load_audio_file(file_path)
    
    if audio is None:
        print(f"  ❌ FAILED to load this file. Skipping...")
        results.append({
            "filename": filename,
            "expected": expected_emotion,
            "status": "FAILED TO LOAD",
            "predicted_emotion": "N/A",
            "confidence": 0.0,
            "correct": False
        })
        continue
    
    print(f"  ✅ Audio loaded successfully")
    print(f"  ⏱️  Duration: {len(audio)/16000:.2f} seconds")
    
    # Pad or truncate
    if len(audio) > MAX_LENGTH:
        print(f"  ✂️  Audio is longer than 5 seconds, truncating...")
        audio = audio[:MAX_LENGTH]
    else:
        audio = np.pad(audio, (0, MAX_LENGTH - len(audio)))
    
    # Extract features
    print("  🔄 Extracting audio features...")
    inputs = feature_extractor(
        audio,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )
    
    # Move to device
    input_values = inputs.input_values.to(device)
    
    # Predict
    print("  🤖 Running model prediction...")
    with torch.no_grad():
        logits = model(input_values).logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_id = torch.argmax(logits, dim=-1).item()
        confidence = probabilities[0][predicted_id].item()
    
    predicted_emotion = ID_TO_LABEL[predicted_id]
    
    # Get all probabilities
    all_probs = {ID_TO_LABEL[j]: probabilities[0][j].item() for j in range(len(EMOTION_LABELS))}
    
    # Check if prediction matches expected
    is_correct = (predicted_emotion == expected_emotion)
    
    # Display results
    print("\n  " + "🎯 PREDICTION RESULT ".center(76, "─"))
    print(f"\n  📋 Expected (from filename): {expected_emotion.upper()}")
    print(f"  🔮 MODEL PREDICTS: {predicted_emotion.upper()}")
    print(f"  📊 Confidence: {confidence * 100:.2f}%")
    
    if is_correct:
        print(f"  ✅ CORRECT! Model got it right!")
    else:
        print(f"  ❌ WRONG! Model predicted {predicted_emotion.upper()} but expected {expected_emotion.upper()}")
    
    print(f"\n  📈 All Emotion Probabilities:")
    # Sort by probability
    sorted_probs = sorted(all_probs.items(), key=lambda x: x[1], reverse=True)
    for emotion, prob in sorted_probs:
        bar_length = int(prob * 50)
        bar = "█" * bar_length + "░" * (50 - bar_length)
        marker = "👈 PREDICTED" if emotion == predicted_emotion else ""
        if emotion == expected_emotion and emotion != predicted_emotion:
            marker = "← EXPECTED"
        elif emotion == expected_emotion and emotion == predicted_emotion:
            marker = "👈 PREDICTED (CORRECT!)"
        print(f"     {emotion.capitalize():12s} {prob*100:6.2f}% │{bar}│ {marker}")
    
    # Store results
    results.append({
        "filename": filename,
        "expected": expected_emotion,
        "status": "SUCCESS",
        "predicted_emotion": predicted_emotion,
        "confidence": confidence * 100,
        "correct": is_correct,
        "all_probabilities": all_probs
    })
    
    print()

# ══════════════════════════════════════════════════════════════════════════════
# SUMMARY OF PREDICTIONS
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("SUMMARY - PREDICTION RESULTS")
print("=" * 80)

successful = [r for r in results if r["status"] == "SUCCESS"]
correct_predictions = [r for r in successful if r["correct"]]
wrong_predictions = [r for r in successful if not r["correct"]]

print(f"\n✅ Successfully tested: {len(successful)}/4 files")
print(f"✅ Correct predictions: {len(correct_predictions)}/{len(successful)}")
print(f"❌ Wrong predictions: {len(wrong_predictions)}/{len(successful)}")

if len(successful) > 0:
    accuracy = (len(correct_predictions) / len(successful)) * 100
    print(f"📊 Accuracy: {accuracy:.2f}%")

# Show all predictions
print(f"\n{'=' * 80}")
print("DETAILED RESULTS")
print('=' * 80)

for i, result in enumerate(results, 1):
    if result['status'] == 'SUCCESS':
        status_icon = "✅" if result['correct'] else "❌"
        print(f"\n{i}. {result['filename']}")
        print(f"   Expected: {result['expected'].upper()}")
        print(f"   Predicted: {result['predicted_emotion'].upper()} (Confidence: {result['confidence']:.2f}%)")
        print(f"   {status_icon} {'CORRECT' if result['correct'] else 'WRONG'}")
    else:
        print(f"\n{i}. {result['filename']}")
        print(f"   ❌ Status: {result['status']}")

# Show wrong predictions in detail
if wrong_predictions:
    print(f"\n{'=' * 80}")
    print("❌ MODEL MISTAKES - WRONG PREDICTIONS")
    print('=' * 80)
    
    for result in wrong_predictions:
        print(f"\n❌ File: {result['filename']}")
        print(f"   Expected emotion: {result['expected'].upper()}")
        print(f"   ➤ Model predicted: {result['predicted_emotion'].upper()} (WRONG!)")
        print(f"   ✓ Correct answer was: {result['expected'].upper()}")
        print(f"   📊 Model was {result['confidence']:.2f}% confident (but wrong)")
        print(f"   → Model confused '{result['expected']}' for '{result['predicted_emotion']}'")

# ══════════════════════════════════════════════════════════════════════════════
# SAVE RESULTS
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("SAVING RESULTS")
print("=" * 80)

output_file = "unknown_dataset_predictions.txt"
with open(output_file, 'w') as f:
    f.write("=" * 80 + "\n")
    f.write("UNKNOWN DATASET - PREDICTION RESULTS\n")
    f.write("=" * 80 + "\n\n")
    
    for i, result in enumerate(results, 1):
        f.write(f"\n{'=' * 80}\n")
        f.write(f"FILE {i}: {result['filename']}\n")
        f.write('=' * 80 + "\n")
        f.write(f"Expected (from filename): {result['expected'].upper()}\n")
        
        if result['status'] == 'SUCCESS':
            f.write(f"PREDICTED EMOTION: {result['predicted_emotion'].upper()}\n")
            f.write(f"CONFIDENCE: {result['confidence']:.2f}%\n")
            f.write(f"RESULT: {'✅ CORRECT' if result['correct'] else '❌ WRONG'}\n")
            f.write(f"\nALL PROBABILITIES:\n")
            sorted_probs = sorted(result['all_probabilities'].items(), key=lambda x: x[1], reverse=True)
            for emotion, prob in sorted_probs:
                f.write(f"  {emotion.capitalize():12s}: {prob*100:6.2f}%\n")
        else:
            f.write(f"STATUS: {result['status']}\n")
        
        f.write("\n")
    
    # Summary
    f.write("\n" + "=" * 80 + "\n")
    f.write("SUMMARY\n")
    f.write("=" * 80 + "\n")
    f.write(f"Total files tested: {len(results)}\n")
    f.write(f"Successful predictions: {len(successful)}\n")
    f.write(f"Correct predictions: {len(correct_predictions)}/{len(successful)}\n")
    f.write(f"Wrong predictions: {len(wrong_predictions)}/{len(successful)}\n")
    if len(successful) > 0:
        f.write(f"Accuracy: {(len(correct_predictions) / len(successful)) * 100:.2f}%\n")

print(f"✅ Results saved to: {output_file}")

# ══════════════════════════════════════════════════════════════════════════════
# FINAL MESSAGE
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("🎉 TESTING COMPLETE!")
print("=" * 80)

if len(successful) > 0:
    if len(correct_predictions) == len(successful):
        print("\n🎉🎉🎉 PERFECT! The model got ALL predictions correct!")
    elif len(correct_predictions) > 0:
        print(f"\n✅ Model got {len(correct_predictions)}/{len(successful)} correct")
        print(f"❌ Model got {len(wrong_predictions)}/{len(successful)} wrong")
    else:
        print("\n❌ The model got all predictions wrong")
        print("   This might indicate the model needs more training or the audio is very different from training data")

print(f"\n💾 Detailed results saved to: {output_file}")
print("\n" + "=" * 80)


Checking model directory...
✅ Directory exists: /kaggle/working/emotion-recognition-model

Contents of /kaggle/working/emotion-recognition-model:
------------------------------------------------------------
  📁 checkpoint-2847/
     - config.json
     - model.safetensors
     - optimizer.pt
     - rng_state.pth
     - scaler.pt
     ... and 3 more files
  📁 checkpoint-3285/
     - config.json
     - model.safetensors
     - optimizer.pt
     - rng_state.pth
     - scaler.pt
     ... and 3 more files
  📁 final/
     - config.json
     - model.safetensors
     - preprocessor_config.json
     - training_args.bin
  📁 runs/
     - Jan29_17-16-50_d701060de0c2

Checking for required model files:
------------------------------------------------------------
  ❌ config.json (MISSING!)
  ❌ preprocessor_config.json (MISSING!)
  ❌ model.safetensors (MISSING!)

✅ Found 'final' subdirectory!

Contents of /kaggle/working/emotion-recognition-model/final:
------------------------------------------------